# Import et processing des données des différentes requêtes - partie 2 geoloc

Objectif : intégrer l'information de géolocalisation aux tweets requêtés et filtrer pour ne retenir uniquement les tweets de la bonne zone géographique. Actuellement le script ne traite que les tweets issus de la requête francophone cash et cb. 

In [24]:
from functools import reduce
import random
import re
import time
from gensim.utils import deaccent
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = None
pd.options.display.max_columns = 30
#from wordcloud import WordCloud
import wordcloud
from nltk.corpus import stopwords
import seaborn as sns
import os

from utils import clean_users_with_geoloc

In [12]:
users = pd.read_csv(f"/home/cash/scripts/geolocation/users_profiles_with_geoloc.csv")

## Data processing et comparaison de volume lors de cette étape

In [13]:
users = clean_users_with_geoloc(users)

In [11]:
users

,user_id,location_profile,country_profile,osm_value_profile,name_profile,county_profile,state_profile,type_profile
0,1164377040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,208433335,nantes france,France,city,Nantes,Loire-Atlantique,Pays de la Loire,city
2,70210292,nevers,France,town,Nevers,Nièvre,Bourgogne-Franche-Comté,city
3,3001695178,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,912871723,kelowna colombie britanique,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
186611,593114226,toulon france,France,city,Toulon,Var,Provence-Alpes-Côte d'Azur,city
186612,4032484462,biot france,France,village,Le Biot,Haute-Savoie,Auvergne-Rhône-Alpes,city
186613,849307944420356096,paris france,France,administrative,Paris,NaN,Île-de-France,county
186614,547048541,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Requête cash : ajout de variables utiles pour analyse et filtrage sur geoloc

In [15]:
df_cash = pd.read_pickle(f"/home/cash/output/cleaned_queries/without_geoloc/query_fr_cash_cleaned.pickle")
# df_cash['covid'] = df_cash['clean_tweet'].str.contains("covid|pand(e|é)mie|(é|e)pid(é|e)mie|corona(virus| virus|-virus)", na = False)
df_cash = pd.merge(df_cash, users, how="left", on="user_id")

In [37]:
# Nombre de tweets hors France (ou geoloc bruit)
nb_tweets_notfrance_or_bruit = len(df_cash[(df_cash["country_profile"] != "France") & (df_cash["country_profile"].isna()==False)])
print("Nombre de tweets hors France : "+str(nb_tweets_notfrance_or_bruit))

# Nombre de tweets France (normalement pas de geoloc bruit car nettoyage avec is_clean_geoloc)
nb_tweets_france = len(df_cash[(df_cash["country_profile"] == "France")  
                  & (df_cash["country_profile"].isna()==False)])
print("Nombre de tweets France : " + str(nb_tweets_france))

# Nombre de tweets geolocalisés (sans les tweets avec geoloc France mais qui sont en fait du bruit)
nb_tweets_geoloc = nb_tweets_notfrance_or_bruit+nb_tweets_france
print("Nombre de tweets geolocalisés : " + str(nb_tweets_geoloc))
print(" Sur les tweets géolocalisés, " +str((nb_tweets_geoloc-nb_tweets_notfrance_or_bruit)/nb_tweets_geoloc) + " concernent la France")

# Nombre de tweets totaux 
nb_tweets = len(pd.read_pickle(f"/home/cash/data/query_fr_cash_cleaned.pickle"))
print("Nombre de tweets total: " + str(nb_tweets))
print(str((nb_tweets-nb_tweets_geoloc)/nb_tweets) + " des tweets sont geolocalisés. ")

# répartition par pays à faire

Nombre de tweets hors France : 39823
Nombre de tweets France : 70363
Nombre de tweets geolocalisés : 110186
 Sur les tweets géolocalisés, 0.6385838491278384 concernent la France
Nombre de tweets total: 200209
0.4496451208487131 des tweets sont geolocalisés. 


In [38]:
# Filtre sur la géoloc : garde que les tweets avec geoloc France
df_cash = df_cash[(df_cash["country_profile"] == "France")]

In [39]:
len_df_cash_raw = len(pd.read_csv(f"/home/cash/data/query_fr_cash.csv"))
len_df_cash_cleaned = len(pd.read_pickle(f"/home/cash/data/query_fr_cash_cleaned.pickle"))
len_df_cash_cleaned_and_geoloc = len(df_cash)

In [40]:
print("Requête cash : avant nettoyage de données : "+ str(len_df_cash_raw))
print("Requête cash : après nettoyage de données : "+ str(len_df_cash_cleaned))
print("Requête cash : après nettoyage de données : "+ 
      str((len_df_cash_raw-len_df_cash_cleaned)/len_df_cash_raw)+
      " des lignes ont été supprimées")
print("Requête cash : après nettoyage de données et filtre sur les localisations : "+ str(len_df_cash_cleaned_and_geoloc))
print("Requête cash : après nettoyage de données et filtre sur les localisations : "+ 
      str((len_df_cash_raw-len_df_cash_cleaned_and_geoloc)/len_df_cash_raw)+
      " des lignes ont été supprimées")

Requête cash : avant nettoyage de données : 314539
Requête cash : après nettoyage de données : 200209
Requête cash : après nettoyage de données : 0.36348433739536273 des lignes ont été supprimées
Requête cash : après nettoyage de données et filtre sur les localisations : 70363
Requête cash : après nettoyage de données et filtre sur les localisations : 0.7762980107395268 des lignes ont été supprimées


Requête cb

In [16]:
df_cb = pd.read_pickle(f"/home/cash/output/cleaned_queries/without_geoloc/query_fr_cb_cleaned.pickle")
# df_cb['covid'] = df_cb['clean_tweet'].str.contains("covid|pand(e|é)mie|(é|e)pid(é|e)mie|corona(virus| virus|-virus)", na = False)
df_cb = pd.merge(df_cb, users, how="left", on="user_id")

In [42]:
# Nombre de tweets hors France (ou geoloc bruit)
nb_tweets_notfrance_or_bruit = len(df_cb[(df_cb["country_profile"] != "France") & (df_cb["country_profile"].isna()==False)])
print("Nombre de tweets hors France : "+str(nb_tweets_notfrance_or_bruit))

# Nombre de tweets France (normalement pas de geoloc bruit car nettoyage avec is_clean_geoloc)
nb_tweets_france = len(df_cb[(df_cb["country_profile"] == "France")  
                  & (df_cb["country_profile"].isna()==False)])
print("Nombre de tweets France : " + str(nb_tweets_france))

# Nombre de tweets geolocalisés (sans les tweets avec geoloc France mais qui sont en fait du bruit)
nb_tweets_geoloc = nb_tweets_notfrance_or_bruit+nb_tweets_france
print("Nombre de tweets geolocalisés (France et autre): " + str(nb_tweets_geoloc))
print(" Sur les tweets géolocalisés, " +str((nb_tweets_geoloc-nb_tweets_notfrance_or_bruit)/nb_tweets_geoloc) + " concernent la France")

# Nombre de tweets totaux 
nb_tweets = len(df_cb)
print("Nombre de tweets total: " + str(nb_tweets))
print(str((nb_tweets-nb_tweets_geoloc)/nb_tweets) + " des tweets sont geolocalisés. ")

Nombre de tweets hors France : 41977
Nombre de tweets France : 104177
Nombre de tweets geolocalisés (France et autre): 146154
 Sur les tweets géolocalisés, 0.7127892496955266 concernent la France
Nombre de tweets total: 255702
0.4284205833352887 des tweets sont geolocalisés. 


In [44]:
# Filtre sur la géoloc : garde que les tweets avec geoloc France
df_cb = df_cb[(df_cb["country_profile"] == "France")]

In [45]:
len_df_cb_raw = len(pd.read_csv(f"/home/cash/data/query_fr_cb.csv"))
len_df_cb_cleaned = len(pd.read_pickle(f"/home/cash/data/query_fr_cb_cleaned.pickle"))
len_df_cb_cleaned_and_geoloc = len(df_cb)

In [46]:
print("Requête cb : avant nettoyage de données : "+ str(len_df_cb_raw))
print("Requête cb : après nettoyage de données : "+ str(len_df_cb_cleaned))
print("Requête cb : après nettoyage de données : "+ 
      str((len_df_cb_raw-len_df_cb_cleaned)/len_df_cb_raw)+
      " des lignes ont été supprimées")
print("Requête cb : après nettoyage de données et filtre sur les localisations : "+ str(len_df_cb_cleaned_and_geoloc))
print("Requête cb : après nettoyage de données et filtre sur les localisations : "+ 
      str((len_df_cb_raw-len_df_cb_cleaned_and_geoloc)/len_df_cb_raw)+
      " des lignes ont été supprimées")

Requête cb : avant nettoyage de données : 294417
Requête cb : après nettoyage de données : 255702
Requête cb : après nettoyage de données : 0.13149716218832472 des lignes ont été supprimées
Requête cash : après nettoyage de données et filtre sur les localisations : 104177
Requête cash : après nettoyage de données et filtre sur les localisations : 0.6461583400415057 des lignes ont été supprimées


Export pickle

In [47]:
df_cash.to_pickle("/home/cash/output/cleaned_queries/with_geoloc/query_fr_cash_cleaned_geoloc.pickle")
df_cb.to_pickle("/home/cash/output/cleaned_queries/with_geoloc/query_fr_cb_cleaned_geoloc.pickle")

In [17]:
print(len(pd.read_pickle(f"/home/cash/output/cleaned_queries/with_geoloc/query_fr_cash_cleaned_geoloc.pickle")))
len(pd.read_pickle(f"/home/cash/output/cleaned_queries/with_geoloc/query_fr_cb_cleaned_geoloc.pickle"))

70363


104177

# Clean geolocalisation des requetes sans contact

In [40]:
users = pd.read_csv(f'/home/cash/data/geolocation/users_profiles_contacless_fr_with_geoloc.csv')

## Data processing et comparaison de volume lors de cette étape

In [41]:
users = clean_users_with_geoloc(users)

In [42]:
df_contacless = pd.read_pickle(f"/home/cash/output/cleaned_queries/without_geoloc/query_fr_contactless_cleaned.pickle")
# df_cash['covid'] = df_cash['clean_tweet'].str.contains("covid|pand(e|é)mie|(é|e)pid(é|e)mie|corona(virus| virus|-virus)", na = False)
df_contacless = pd.merge(df_contacless, users, how="left", on="user_id")

In [43]:
# Nombre de tweets hors France (ou geoloc bruit)
nb_tweets_notfrance_or_bruit = len(df_contacless[(df_contacless["country_profile"] != "France") & (df_contacless["country_profile"].isna()==False)])
print("Nombre de tweets hors France : "+str(nb_tweets_notfrance_or_bruit))
print(df_contacless.shape)

# Nombre de tweets France (normalement pas de geoloc bruit car nettoyage avec is_clean_geoloc)
nb_tweets_france = len(df_contacless[(df_contacless["country_profile"] == "France")  
                  & (df_contacless["country_profile"].isna()==False)])
print("Nombre de tweets France : " + str(nb_tweets_france))

# Nombre de tweets geolocalisés (sans les tweets avec geoloc France mais qui sont en fait du bruit)
nb_tweets_geoloc = nb_tweets_notfrance_or_bruit+nb_tweets_france
print("Nombre de tweets geolocalisés : " + str(nb_tweets_geoloc))
print(" Sur les tweets géolocalisés, " +str((nb_tweets_geoloc-nb_tweets_notfrance_or_bruit)/nb_tweets_geoloc) + " concernent la France")

# Nombre de tweets totaux 
nb_tweets = len(df_contacless)
print("Nombre de tweets total: " + str(nb_tweets))
print(str((nb_tweets-nb_tweets_geoloc)/nb_tweets) + " des tweets sont geolocalisés. ")


Nombre de tweets hors France : 7341
(48030, 49)
Nombre de tweets France : 23336
Nombre de tweets geolocalisés : 30677
 Sur les tweets géolocalisés, 0.760700198846041 concernent la France
Nombre de tweets total: 48030
0.3612950239433687 des tweets sont geolocalisés. 


## Filtre : sélection des tweets géolocalisés

In [44]:
# Filtre sur la géoloc : garde que les tweets avec geoloc France
df_contacless = df_contacless[(df_contacless["country_profile"] == "France")]

In [45]:
df_contacless.shape

(23336, 49)

In [33]:
df_contactless_cb_fr = pd.read_csv(f"/home/cash/data/queries/query_contactless_cb_fr_part1.csv")
df_contactless_smartphone_fr = pd.read_csv(f"/home/cash/data/queries/query_contactless_smartphone_fr_part1.csv")
len_df_contactless_raw = len(pd.concat([df_contactless_cb_fr, df_contactless_smartphone_fr]))
len_df_contactless_cleaned = len(pd.read_pickle(f"/home/cash/output/cleaned_queries/without_geoloc/query_fr_contactless_cleaned.pickle"))
len_df_contactless_cleaned_and_geoloc = len(df_contacless)

In [39]:
print("Requête sans contact : avant nettoyage de données : "+ str(len_df_contactless_raw))
print("Requête sans contact : après nettoyage de données : "+ str(len_df_contactless_cleaned))
print("Requête sans contact : après nettoyage de données : "+ 
      str((len_df_contactless_raw-len_df_contactless_cleaned)/len_df_contactless_raw)+
      " des lignes ont été supprimées")
print("Requête sans contact : après nettoyage de données et filtre sur les localisations : "+ str(len_df_contactless_cleaned_and_geoloc))
print("Requête sans contact : après nettoyage de données et filtre sur les localisations : "+ 
      str((len_df_contactless_raw-len_df_contactless_cleaned_and_geoloc)/len_df_contactless_raw)+
      " des lignes ont été supprimées")

Requête sans contact : avant nettoyage de données : 101236
Requête sans contact : après nettoyage de données : 48030
Requête sans contact : après nettoyage de données : 0.5255640286064246 des lignes ont été supprimées
Requête sans contact : après nettoyage de données et filtre sur les localisations : 23336
Requête sans contact : après nettoyage de données et filtre sur les localisations : 0.7694891145442333 des lignes ont été supprimées


### Ecriture du pickle : table nettoyée + geolocalisée

In [46]:
df_contacless.to_pickle("/home/cash/output/cleaned_queries/with_geoloc/query_fr_contactless_cleaned_geoloc.pickle")

In [47]:
# Vérification
print(len(pd.read_pickle(f"/home/cash/output/cleaned_queries/with_geoloc/query_fr_contactless_cleaned_geoloc.pickle")))

23336
